# Process synthetic enhancers sequencing data
# = BC quantification

Run on one of the Genius nodes to match the modules.

In [2]:
cd /staging/leuven/stg_00002/lcb/lminn/Enhancer_design/MPRA/CHEQ-seq_cloning2/BC_quantification_20201006

## 00. Combine raw reads and 01. Fastqc

- Create within your folder the 00.raw_reads folder where you create a text file CHEQseq_synth_names.txt.
- Within the text file put the names of the samples, e.g. CSE__8636ca__CheqSeq_cDNA_MM001_2000_enh_pool_Liesbeth. Make sure to put a line symbol at the end of the text file so that also the last sample is processed.
- Provide the sequencing run paths 
- The script will then combine the same samples from different runs within 1 fastq
- Make sure to check if you want to use R1 or R2, in this case R2 is the RC of R1, but we will use R1 because the script is made for this (the rBCs will be not be RC for instance in R1)


__FYI: We have now only 1 run so the script will just put those fastqs within the 00.raw_reads folder; except for the MM029 where we have two runs__

In [3]:
mkdir 01.fastqc
mkdir 02.cleaned_reads
mkdir 03.sequencing_saturation
mkdir 10.bc_count

mkdir: cannot create directory ‘01.fastqc’: File exists
mkdir: cannot create directory ‘02.cleaned_reads’: File exists
mkdir: cannot create directory ‘03.sequencing_saturation’: File exists
mkdir: cannot create directory ‘10.bc_count’: File exists


: 1

In [4]:
module load FastQC/0.11.8-Java-1.8.0_162
# For group of samples
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do 
# Path to the different runs
NovaSeq6000_20201006="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20201006/Demultiplexed/${line%Cheq*}*R1*"
NovaSeq6000_20201013="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20201013/Demultiplexed/${line%Cheq*}*R1*"
cat ${NovaSeq6000_20201006} ${NovaSeq6000_20201013} > 00.raw_reads/${line}_R1.fastq.gz
fastqc -t 10 00.raw_reads/${line}_R1.fastq.gz -o 01.fastqc
done

cat: /staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20201013/Demultiplexed/CSE__6f8b76__*R1*: No such file or directory
Started analysis of CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA_R1.fastq.gz
Approx 5% complete for CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA_R1.fastq.gz
Approx 10% complete for CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA_R1.fastq.gz
Approx 15% complete for CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA_R1.fastq.gz
Approx 20% complete for CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA_R1.fastq.gz
Approx 25% complete for CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA_R1.fastq.gz
Approx 30% complete for CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA_R1.fastq.gz
Approx 35% complete for CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA_R1.fastq.gz
Approx 40% complete for CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM00

In [5]:
# Total number of reads for R1 per sample over the runs
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
echo -e "Combined number of reads for ${line}: " $(( $(zcat 00.raw_reads/${line}_R1.fastq.gz | wc -l) / 4 ))
done

Combined number of reads for CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA:  8457241
Combined number of reads for CSE__81ba64__CheqSeq_Liesbeth_specific_enhancer_pool_MM074_cDNA:  8626303
Combined number of reads for CSE__839e9e__CheqSeq_Liesbeth_specific_enhancer_pool_MM087_cDNA:  5335676
Combined number of reads for CSE__7cb57c__CheqSeq_Liesbeth_specific_enhancer_pool_MM029_cDNA:  1516983
Combined number of reads for CSE__8f7a3e__CheqSeq_Liesbeth_specific_enhancer_pool_MM099_cDNA:  4266283
Combined number of reads for CSE__8b9ec1__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_plasmid:  14868109
Combined number of reads for CSE__cd9463__CheqSeq_Liesbeth_specific_enhancer_pool_MM074_plasmid:  16533113
Combined number of reads for CSE__0bf4ec__CheqSeq_Liesbeth_specific_enhancer_pool_MM087_plasmid:  13382837
Combined number of reads for CSE__f7b5af__CheqSeq_Liesbeth_specific_enhancer_pool_MM029_plasmid:  13441105
Combined number of reads for CSE__0bd121__CheqSeq_Liesbet

## 02. Generate clean reads

In [6]:
module use /data/leuven/software/biomed/skylake_centos7/2018a/modules/all/
module load SeqKit/0.10.2
module load cutadapt/1.18-foss-2018a-Python-3.7.0
start=`date +%s`
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
    # Do cutadapt trimming on R1 reads
        # Infor form David: Adjust -e to 0.2 for the NovaSeq run because of low sequencing quality on 5 bases! 
        # In our case good quality except for the first 3bp of one of the samples -> keep the max error rate at 15%
        # However, only bewteen 61% and 89% of the reads are written
cutadapt -g ACTCGAGCCGAGCAGGCGCGCCGATC...GGACCGG -e 0.2 -l 17 -m 17 -j 10 --discard-untrimmed \
         -o 02.cleaned_reads/${line}_R1.fastq.gz \
         00.raw_reads/${line}_R1.fastq.gz
done
end=`date +%s`
echo $((end-start))


## This is if it is R2 (the also RC)
            # Do cutadapt trimming on R2 reads and then reverse complement
        #cutadapt -g GCGAATTAATTCGGGCCCCGGTCC...GATCGGCGCGCCTGC -e 0.15 -l 17 -m 17 -j 10 --discard-untrimmed \
        #         00.raw_reads/${line}_R2.fastq.gz | \
        #seqkit seq -r -p -o 02.cleaned_reads/${line}_R2.fastq.gz

Python/3.7.0-foss-2018a(38):ERROR:150: Module 'Python/3.7.0-foss-2018a' conflicts with the currently loaded module(s) 'Python/3.7.4-foss-2018a'
Python/3.7.0-foss-2018a(38):ERROR:102: Tcl command execution failed: conflict Python

This is cutadapt 1.18 with Python 3.7.4
Command line parameters: -g ACTCGAGCCGAGCAGGCGCGCCGATC...GGACCGG -e 0.2 -l 17 -m 17 -j 10 --discard-untrimmed -o 02.cleaned_reads/CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA_R1.fastq.gz 00.raw_reads/CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA_R1.fastq.gz
Processing reads on 10 cores in single-end mode ...
Finished in 38.85 s (5 us/read; 13.06 M reads/minute).

=== Summary ===

Total reads processed:               8,457,241
Reads with adapters:                 8,422,072 (99.6%)
Reads that were too short:              97,711 (1.2%)
Reads written (passing filters):     8,324,361 (98.4%)

Total basepairs processed:   414,404,809 bp
Total written (filtered):    141,514,137 bp (34.1%)

=

In [7]:
module load fastp/0.20.0-foss-2018a
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
# Combine R1 and R2 
#zcat 02.cleaned_reads/${line}_R*.fastq.gz | \

#  Filter for Q30 reads
zcat 02.cleaned_reads/${line}_R1.fastq.gz | fastp --stdin -e 30 -h /dev/null/fastp.html -j /dev/null/fastp.json -w 8 \
      -o 02.cleaned_reads/${line}_R1_q30.fastq.gz
done

Adapter auto-detection is disabled for STDIN mode
Read1 before filtering:
total reads: 8324361
total bases: 141514137
Q20 bases: 139248360(98.3989%)
Q30 bases: 135548709(95.7846%)

Read1 after filtering:
total reads: 8111342
total bases: 137892814
Q20 bases: 136812785(99.2168%)
Q30 bases: 133755537(96.9996%)

Filtering result:
reads passed filter: 8111342
reads failed due to low quality: 213019
reads failed due to too many N: 0
reads failed due to too short: 0
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate (may be overestimated since this is SE data): 0%

JSON report: /dev/null/fastp.json
HTML report: /dev/null/fastp.html

fastp --stdin -e 30 -h /dev/null/fastp.html -j /dev/null/fastp.json -w 8 -o 02.cleaned_reads/CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA_R1_q30.fastq.gz 
fastp v0.20.0, time used: 33 seconds
Adapter auto-detection is disabled for STDIN mode
Read1 before filtering:
total reads: 8263521
total bases: 140479857
Q20 ba

In [8]:
# Total number of clean reads per sample
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
echo -e "Number of reads after Q30 filtering for ${line}: " $(( $(zcat 02.cleaned_reads/${line}_R1_q30.fastq.gz | wc -l) / 4 ))
done

Number of reads after Q30 filtering for CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA:  8111342
Number of reads after Q30 filtering for CSE__81ba64__CheqSeq_Liesbeth_specific_enhancer_pool_MM074_cDNA:  7992966
Number of reads after Q30 filtering for CSE__839e9e__CheqSeq_Liesbeth_specific_enhancer_pool_MM087_cDNA:  4971234
Number of reads after Q30 filtering for CSE__7cb57c__CheqSeq_Liesbeth_specific_enhancer_pool_MM029_cDNA:  1203434
Number of reads after Q30 filtering for CSE__8f7a3e__CheqSeq_Liesbeth_specific_enhancer_pool_MM099_cDNA:  3952322
Number of reads after Q30 filtering for CSE__8b9ec1__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_plasmid:  14267169
Number of reads after Q30 filtering for CSE__cd9463__CheqSeq_Liesbeth_specific_enhancer_pool_MM074_plasmid:  15434299
Number of reads after Q30 filtering for CSE__0bf4ec__CheqSeq_Liesbeth_specific_enhancer_pool_MM087_plasmid:  12540334
Number of reads after Q30 filtering for CSE__f7b5af__CheqSeq_Liesbeth_specif

## 03. Sequencing saturarion

- First extract the rBCs from the fastq and put it in a text file
- To calculate the statureation, subsample this rBC text file (choose the subsampling frequency a bit depending on the data) and print the number of cleaned reads (i.e. rBCs) taken and the number of unqiue ones for that subsample.
    - e.g. now we have in the 03.sequencing_saturation/${line}_R1_q30.txt files a max of 20,884,696 
- You can then plot these numbers in excell in a line graph to assess the saturation

In [9]:
# Extract DNA sequences from fastq
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
zcat 02.cleaned_reads/${line}_R1_q30.fastq.gz | sed -n '2~4p' > 03.sequencing_saturation/${line}_R1_q30.txt
done

__Number of barcodes and unique barcodes__

In [10]:
# Count total number of rBC in files
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
echo 03.sequencing_saturation/${line}_R1_q30.txt && cat 03.sequencing_saturation/${line}_R1_q30.txt | wc -l
done

# Count total number of UNIQUE rBC in files
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
echo 03.sequencing_saturation/${line}_R1_q30.txt && cat 03.sequencing_saturation/${line}_R1_q30.txt | sort | uniq | wc -l
done

03.sequencing_saturation/CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA_R1_q30.txt
8111342
03.sequencing_saturation/CSE__81ba64__CheqSeq_Liesbeth_specific_enhancer_pool_MM074_cDNA_R1_q30.txt
7992966
03.sequencing_saturation/CSE__839e9e__CheqSeq_Liesbeth_specific_enhancer_pool_MM087_cDNA_R1_q30.txt
4971234
03.sequencing_saturation/CSE__7cb57c__CheqSeq_Liesbeth_specific_enhancer_pool_MM029_cDNA_R1_q30.txt
1203434
03.sequencing_saturation/CSE__8f7a3e__CheqSeq_Liesbeth_specific_enhancer_pool_MM099_cDNA_R1_q30.txt
3952322
03.sequencing_saturation/CSE__8b9ec1__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_plasmid_R1_q30.txt
14267169
03.sequencing_saturation/CSE__cd9463__CheqSeq_Liesbeth_specific_enhancer_pool_MM074_plasmid_R1_q30.txt
15434299
03.sequencing_saturation/CSE__0bf4ec__CheqSeq_Liesbeth_specific_enhancer_pool_MM087_plasmid_R1_q30.txt
12540334
03.sequencing_saturation/CSE__f7b5af__CheqSeq_Liesbeth_specific_enhancer_pool_MM029_plasmid_R1_q30.txt
12508271
03.sequencin

In [11]:
# Subsample, count total and unique reads
start=`date +%s`

cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
file="03.sequencing_saturation/${line}_R1_q30.txt"
nreads=$(cat $file | wc -l)
 # Start by removing parallel instructions file
rm 03.sequencing_saturation/parallel.txt
 # Remove any previoulsy created sat values for that file
rm ${file%_*}_sat.txt
    for i in $(seq 0 200000 $nreads)
    do
     # Print number of total and unique reads for every subsamples
    echo "echo $i \$(shuf -n $(($i)) $file | sort | uniq | wc -l) >> ${file%_*}_sat.txt" \
    >> 03.sequencing_saturation/parallel.txt
    done
cat 03.sequencing_saturation/parallel.txt | parallel -j 16
 # Put values in numerical order
cat ${file%_*}_sat.txt | sort -g > ${file%_*}_satsort.txt && mv ${file%_*}_satsort.txt ${file%_*}_sat.txt
done

end=`date +%s`
echo $((end-start))

559


## 10. Make rBC count matrix and assign to enhancers

In [12]:
# Generate rBC count matrix for each samples
parallel \
"cat 03.sequencing_saturation/{}_R1_q30.txt | sort | uniq -c | awk -F \" \" '{print \$2\"\t\"\$1}' \
   > 10.bc_count/{}_count.txt" ::: $(cat 00.raw_reads/CHEQseq_synth_names.txt)

# Count number of unique rBC per sample
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
echo 10.bc_count/${line}_count.txt && cat 10.bc_count/${line}_count.txt | wc -l
done

10.bc_count/CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA_count.txt
1513297
10.bc_count/CSE__81ba64__CheqSeq_Liesbeth_specific_enhancer_pool_MM074_cDNA_count.txt
1338883
10.bc_count/CSE__839e9e__CheqSeq_Liesbeth_specific_enhancer_pool_MM087_cDNA_count.txt
967709
10.bc_count/CSE__7cb57c__CheqSeq_Liesbeth_specific_enhancer_pool_MM029_cDNA_count.txt
103386
10.bc_count/CSE__8f7a3e__CheqSeq_Liesbeth_specific_enhancer_pool_MM099_cDNA_count.txt
481129
10.bc_count/CSE__8b9ec1__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_plasmid_count.txt
2364178
10.bc_count/CSE__cd9463__CheqSeq_Liesbeth_specific_enhancer_pool_MM074_plasmid_count.txt
2370333
10.bc_count/CSE__0bf4ec__CheqSeq_Liesbeth_specific_enhancer_pool_MM087_plasmid_count.txt
2267394
10.bc_count/CSE__f7b5af__CheqSeq_Liesbeth_specific_enhancer_pool_MM029_plasmid_count.txt
1489261
10.bc_count/CSE__0bd121__CheqSeq_Liesbeth_specific_enhancer_pool_MM099_plasmid_count.txt
2849052


__We used the 1x library for transfections__

In [13]:
# Assign rBC to enhancers
    # List of rBC-Enhancer files : I one have 1, I will use the high complexity
    # The file had another order: rBC - enhancer -> adatp it to (1) readID (put .) (2) enhancer (3) rBC ; and 

# OLSA="/staging/leuven/stg_00002/lcb/dwmax/documents/aertslab/CH1/CHEQ-seq_OLS_DavidTileA/20.coupling/CH1__b87bb6__CheqSeq_OLS_David_Pool_A_enhancer-barcode_S2_enhancer_barcode_coupled_wo_empty_vectors_bc_rc.tsv.gz"


cat /user/leuven/317/vsc31703/stg_00002/Enhancer_design/MPRA/CHEQ-seq_cloning2/E-BC_coupling/03.E-BC_coupling/CSE__953401__CHEQseq-cl2_2000enh_1x_enhancerID_sBC_rBC_coupled.format_unique.sorted_1to1.matrix | \
awk '{print "." "\t" $2 "\t" $1 }' | gzip -c > /user/leuven/317/vsc31703/stg_00002/Enhancer_design/MPRA/CHEQ-seq_cloning2/E-BC_coupling/03.E-BC_coupling/CSE__953401__CHEQseq-cl2_2000enh_1x_enhancerID_sBC_rBC_coupled.format_unique.sorted_1to1.matrix.gz

COUPLE="/user/leuven/317/vsc31703/stg_00002/Enhancer_design/MPRA/CHEQ-seq_cloning2/E-BC_coupling/03.E-BC_coupling/CSE__953401__CHEQseq-cl2_2000enh_1x_enhancerID_sBC_rBC_coupled.format_unique.sorted_1to1.matrix.gz"
   
   # OLS A samples
#parallel \
#"awk -F '\t' -v OFS='\t' 'FNR==NR{a[\$3]=\$2 FS \$3;next}{ print a[\$1],\$2}' \
#   <( zcat ${OLSA} ) \
#   <( cat 10.bc_count/{}_count.txt ) \
#   | awk -F '\t' '{if (\$1) print \$0;}' \
#   > 10.bc_count/{}_count_final.txt" ::: $(grep 'OLS_A' 00.raw_reads/CHEQseq_synth_names.txt)
   
   # For mine:
parallel \
"awk -F '\t' -v OFS='\t' 'FNR==NR{a[\$3]=\$2 FS \$3;next}{ print a[\$1],\$2}' \
   <( zcat ${COUPLE} ) \
   <( cat 10.bc_count/{}_count.txt ) \
   | awk -F '\t' '{if (\$1) print \$0;}' \
   > 10.bc_count/{}_count_final.txt" ::: $(cat 00.raw_reads/CHEQseq_synth_names.txt)   
   
 

In [14]:
# Count number of rBC associated to an enhancer
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
echo -e "${line}\t$(cat 10.bc_count/${line}_count_final.txt | wc -l)"
done

CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA	597159
CSE__81ba64__CheqSeq_Liesbeth_specific_enhancer_pool_MM074_cDNA	522569
CSE__839e9e__CheqSeq_Liesbeth_specific_enhancer_pool_MM087_cDNA	384723
CSE__7cb57c__CheqSeq_Liesbeth_specific_enhancer_pool_MM029_cDNA	40027
CSE__8f7a3e__CheqSeq_Liesbeth_specific_enhancer_pool_MM099_cDNA	183189
CSE__8b9ec1__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_plasmid	894810
CSE__cd9463__CheqSeq_Liesbeth_specific_enhancer_pool_MM074_plasmid	887510
CSE__0bf4ec__CheqSeq_Liesbeth_specific_enhancer_pool_MM087_plasmid	861629
CSE__f7b5af__CheqSeq_Liesbeth_specific_enhancer_pool_MM029_plasmid	575865
CSE__0bd121__CheqSeq_Liesbeth_specific_enhancer_pool_MM099_plasmid	907338


In [15]:
# Coverage (i.e number of enhancer recovered) and Median number of rBC per enhancer
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
file="10.bc_count/${line}_count_final.txt"
middle=$(($(awk '{print $1}' $file | sort | uniq -c | wc -l)/2))
echo $line
awk '{print $1}' $file | sort | uniq -c | wc -l
awk '{print $1}' $file | sort | uniq -c | awk '{print $1}' | sort -g | sed -n ${middle}p
done

CSE__6f8b76__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_cDNA
1996
158
CSE__81ba64__CheqSeq_Liesbeth_specific_enhancer_pool_MM074_cDNA
1997
137
CSE__839e9e__CheqSeq_Liesbeth_specific_enhancer_pool_MM087_cDNA
1996
98
CSE__7cb57c__CheqSeq_Liesbeth_specific_enhancer_pool_MM029_cDNA
1866
9
CSE__8f7a3e__CheqSeq_Liesbeth_specific_enhancer_pool_MM099_cDNA
1990
41
CSE__8b9ec1__CheqSeq_Liesbeth_specific_enhancer_pool_MM001_plasmid
1998
254
CSE__cd9463__CheqSeq_Liesbeth_specific_enhancer_pool_MM074_plasmid
1998
253
CSE__0bf4ec__CheqSeq_Liesbeth_specific_enhancer_pool_MM087_plasmid
1998
243
CSE__f7b5af__CheqSeq_Liesbeth_specific_enhancer_pool_MM029_plasmid
1998
153
CSE__0bd121__CheqSeq_Liesbeth_specific_enhancer_pool_MM099_plasmid
1998
260
